In [8]:
!pip install kiwi
!pip install kiwipiepy

### DEEPL API 로 역번역 데이터 노이즈 제거 및 증강

In [ ]:
import pandas as pd
import requests
import time
import csv

# DeepL API 설정
DEEPL_API_KEY = "키 입력 창"  
DEEPL_API_URL = "https://api-free.deepl.com/v2/translate"

# 데이터 불러오기
df = pd.read_csv("/data/ephemeral/data/filtered_texts_test.csv")

# DeepL API를 이용한 역번역 함수
def back_translate_with_deepl(text, source_lang="KO", target_lang="EN"):
    try:
        response = requests.post(
            DEEPL_API_URL,
            data={
                "auth_key": DEEPL_API_KEY,
                "text": text,
                "source_lang": source_lang,
                "target_lang": target_lang
            }
        )
        translated_text = response.json()["translations"][0]["text"]
        time.sleep(1)

        response = requests.post(
            DEEPL_API_URL,
            data={
                "auth_key": DEEPL_API_KEY,
                "text": translated_text,
                "source_lang": target_lang,
                "target_lang": source_lang
            }
        )
        back_translated_text = response.json()["translations"][0]["text"]
        return back_translated_text
    except Exception as e:
        print(f"Error during back translation: {e}")
        return text  # 오류 발생 시 원본 텍스트 반환

# 증강 데이터 저장할 리스트 초기화
augmented_data = []

# 데이터셋 순회하면서 증강 데이터 생성
for idx, row in df.iterrows():
    original_text = row['text']
    label = row['target']
    augmented_data.append({'ID': row['ID'], 'text': original_text, 'target': label})

    # DeepL을 이용한 역번역 증강
    try:
        back_translated_text = back_translate_with_deepl(original_text)
        augmented_data.append({'ID': row['ID'], 'text': back_translated_text, 'target': label})
    except Exception as e:
        print(f"Error in back translation for ID {row['ID']}: {e}")

# 원본 데이터와 증강 데이터를 합쳐 최종 데이터셋 구성
augmented_df = pd.DataFrame(augmented_data)
augmented_df.to_csv(
    "/data/ephemeral/data/train_augmented_fixed.csv",
    index=False,
    quoting=csv.QUOTE_ALL,
    encoding='utf-8'
)


### Deepl 을 사용하면서 데이터셋 형태 및 구조가 약간 바뀌어서 다시 조정하는 과정 추가

In [ ]:
import pandas as pd
import csv

#데이터 경로 입력
input_file = '/data/ephemeral/data/train_augmented.csv'
output_file = '/data/ephemeral/data/train_aug_test.csv'

#데이터 로드
data = pd.read_csv(input_file, encoding='utf-8', engine='python')

# 혹시 없는 데이터셋 에 대해 임시로 채우기
data['text'] = data['text'].fillna('')

#데이터 형태를 모두 str 변환 => DEEPL 번역 과정에서 str 형태로 반환이 안된것 같음
data['text'] = data['text'].astype(str)

#이후 1차 text 컬럼 정리
def clean_text(text):
    text = text.replace(',', ' ')     
    text = text.replace('"', '')     
    text = text.replace("'", '')    
    text = text.replace('\n', ' ')    
    text = text.replace('\r', ' ')    
    text = text.strip()
    return text

data['text'] = data['text'].apply(clean_text)

# 한번 str 인지 체크
non_string_entries = data[~data['text'].apply(lambda x: isinstance(x, str))]
if not non_string_entries.empty:
    print("'text'컬럼 내 str 아닌 데이터 없음:")
    print(non_string_entries)


# 데이터 저장
data.to_csv(output_file, index=False, encoding='utf-8')


In [ ]:
import pandas as pd
import re

data = pd.read_csv('/data/ephemeral/data/train_aug_test.csv')

# 조건에 없는 텍스트 및 공백 등의 조건 조정
def clean_text(text):
    text = str(text) if pd.notna(text) else ""  
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)  
    text = re.sub(r'\s+', ' ', text).strip()  
    return text

# 필터 적용
data['text'] = data['text'].apply(clean_text)

# 결과가 없는 데이터 열 제거
data = data[data['text'] != ""]

data.to_csv('/data/ephemeral/data/train_aug_test_cleaned.csv', index=False)